# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [38]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [51]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [52]:
data.head()
print(len(data))

4870


#### 1. What test is appropriate for this problem? Does CLT apply?

We will use a two-sample z-test for the difference in proportions. CLT applies because our sample size is large (n = 4870) and observations are independent.

#### 2. What are the null and alternate hypotheses?

$H_0$: $P_1$ = $P_2$

$H_A$: $P_1$ $\neq$ $P_2$


In [42]:
w = data[data.race=='w']
b = data[data.race=='b']

#### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [43]:
# compute margin of error, confidence interval and p-value

# white
p_w = sum(w.call) / len(w.call)
w_std_err = np.sqrt((p_w*(1-p_w))/len(w.call))

# black
p_b = sum(b.call) / len(b.call)
b_std_err = np.sqrt((p_b*(1-p_b))/len(b.call))

# 95% confidence
z = 1.96

# margin of error of the difference of proportions
marg_err = z * (np.sqrt(w_std_err ** 2 + b_std_err ** 2))
print("Margin of Error:",marg_err)

# 95% confidence interval
limit_1 = (p_w - p_b) - marg_err
limit_2 = (p_w - p_b) + marg_err
conf_in = (limit_1, limit_2)
print("95% Confidence Interval:",conf_in)

Margin of Error: 0.015255406349886438
95% Confidence Interval: (0.016777447859559147, 0.047288260559332024)


In [44]:
# frequentist test 

# overall sample proportion
p_success = (sum(w.call) + sum(b.call)) / (len(w.call) + len(b.call))

# Z score
z_score = (p_w - p_b - 0) / \
    (np.sqrt( p_success * (1 - p_success) * ( 1 / len(w.call) + 1 / len(b.call))))
print('Z-score:', Z)

# p-value
p_val = stats.norm.sf(z_score) * 2
print('P value:', p_val)

Z-score: 4.108412152434346
P value: 3.983886837585077e-05


The p-value is very low, so we reject the null hypothesis.

In [49]:
# bootstrap approach

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size = 1):
    perm_replicates = np.empty(size)

    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def diff_of_proportion(data_1, data_2):
    return (sum(data_1) / len(data_1) - (sum(data_2) / len(data_2)))

empirical_diff_proportion = diff_of_proportion(w.call, b.call)

perm_replicates = draw_perm_reps(w.call, b.call, 
                                 diff_of_proportion, size = 10000)

p = np.sum(perm_replicates >= empirical_diff_proportion) / len(perm_replicates)
print('Bootstrap p value:', p)

Bootstrap p value: 0.0


#### 4. Write a story describing the statistical significance in the context or the original problem.

In this problem, we tested the hypothesis that race does not have a significant impact on whether or not a resume receives a callback. To do this, we compared the proportions of black-sounding resumes that received a callback to white-sounding resumes that received a callback. 

First, with a confidence interval we found that 95% of the time, the difference between proportions lies between 0.017 and 0.047. Then, with a Z-test we found a p-value of 3.983886837585077e-05, which is low enough to reject the null hypothesis. Finally, we performed a permutation test also giving us a very small p-value.

From our tests we can conclude that race does have a statistically significant impact on resume callbacks. This dataset gives us evidence of racial discrimination in this sample of the United States labor market. 

#### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis does not necessarily imply that race is the most important factor in resume callbacks. We only tested the hypothesis that it has some influence. In order to determine its 'importance', we would have to amend our analysis by testing all the other factors provided in the dataset. 